References:
- What We have tried 1: https://liangfgithub.github.io/Proj/F24_Proj2_hints_1_Python.html

- performance target: 1580 or less; 1pt: above 1680

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns


In [3]:
# import all libraries
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
##Replace with your path to the Proj2_Data folder on GoogleColab
path_to_data='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project2/MyCode/Proj2_Data'


## **Data Exploration**

In [6]:
proj2_data_train = pd.read_csv(f'{path_to_data}/fold_1/train.csv')
proj2_data_test = pd.read_csv(f'{path_to_data}/fold_1/test.csv')

# Preview the first 5 lines of the train loaded data
proj2_data_train




,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
164110,45,98,2011-01-28,77.00,False
164111,45,98,2011-02-04,57.25,False
164112,45,98,2011-02-11,388.80,True
164113,45,98,2011-02-18,284.50,False


In [7]:
# Preview the first 5 lines of the test loaded data
proj2_data_test.head()

,Store,Dept,Date,IsHoliday
0,1,1,2011-03-04,False
1,1,1,2011-03-11,False
2,1,1,2011-03-18,False
3,1,1,2011-03-25,False
4,1,1,2011-04-01,False


In [8]:
proj2_data_train.shape

(164115, 5)

In [9]:
proj2_data_test.shape

(26559, 4)

In [10]:
proj2_data_train.isnull().sum()

,0
Store,0
Dept,0
Date,0
Weekly_Sales,0
IsHoliday,0


In [11]:
proj2_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164115 entries, 0 to 164114
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         164115 non-null  int64  
 1   Dept          164115 non-null  int64  
 2   Date          164115 non-null  object 
 3   Weekly_Sales  164115 non-null  float64
 4   IsHoliday     164115 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 5.2+ MB


In [12]:
# train_y = proj2_data_train['Weekly_Sales']
# train = proj2_data_train.drop('Weekly_Sales', axis=1, inplace=True)
# train=proj2_data_train


In [13]:
# train.describe(include='all')


*Produce* a pair-wise scatter plot. Caution: a big figure.


In [14]:
# sns.pairplot(proj2_data_train)


##Approach I
I began with a simplistic strategy: using the sales data from the most recent week to forecast all subsequent weekly sales. For instance, during the first iteration (t=1), our objective is to forecast sales for March and April of 2011. For every store-department combination, I referenced the sales data from the final week of February 2011 and used it as a basis for predicting sales throughout March and April of 2011.

If any data from that particular week is absent, I substitute the missing values with zeros.

Below is my code for a particular fold.

In [15]:
num_folds = 10
for i in range(num_folds):
    train = pd.read_csv(f'{path_to_data}/fold_{i+1}/train.csv')
    test = pd.read_csv(f'{path_to_data}/fold_{i+1}/test.csv')

    most_recent_date = train['Date'].max()

    # Filter and select necessary columns
    tmp_train = train[train['Date'] == most_recent_date].copy()
    tmp_train.rename(columns={'Weekly_Sales': 'Weekly_Pred'}, inplace=True)
    tmp_train = tmp_train.drop(columns=['Date', 'IsHoliday'])

    # Left join with the test data
    test_pred = test.merge(tmp_train, on=['Dept', 'Store'], how='left')

    # Fill NaN values with 0 for the Weekly_Pred column
    test_pred['Weekly_Pred'].fillna(0, inplace=True)

    # Write the output to CSV
    test_pred.to_csv(f'{path_to_data}/fold_{i+1}/mypred.csv', index=False)

After running the script in each of the 10 folders, I generated a mypred.csv file in each folder.

Next, I used the following function to calculate the weighted mean absolute error. The same function will be used to evaluate your performance.

Within the function, I first load the complete test data along with the labels. Then, I loop through each of the 10 folds, extracting the corresponding subset based on the test range for that fold, and compute the performance.

In [16]:
def myeval():
    test_with_label = pd.read_csv(f'{path_to_data}/test_with_label.csv')
    num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'{path_to_data}/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'{path_to_data}/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)
        test_pred = test_pred.drop(columns=['IsHoliday'])

        new_test = test.merge(test_pred, on=['Date', 'Store', 'Dept'], how='left')

        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

Below are the errors across 10 folds.



In [17]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")


	2078.726
	2589.338
	2253.936
	2823.098
	5156.012
	4218.348
	2269.904
	2143.839
	2221.145
	2372.425
2812.677


## Approach II
The initial approach did not yield satisfactory results. A closer inspection of the time series plots for various store and department combinations led us to refine our model. Now, for each week we're predicting, we'll leverage data from the corresponding week in the previous year.

By "corresponding week", we introduce a new variable, "Wk", to numerically represent each week of the year, ranging from 1 to 52 (or occasionally 53). For instance, when predicting sales for Week 20 of 2011, we'd consult the data from Week 20 of 2010.

In R, we encountered a misalignment issue: the final week of 2010 is labeled as Week 53, whereas 2011's is Week 52. This discrepancy is particularly problematic since both weeks usually represent the Christmas holidays.

In Python, we are lucky: within our specific time range, each year consistently consists of 52 weeks and the weeks align perfectly:

Wk = 6, Super Bowl
Wk = 36, Labor Day
Wk = 47, Thanksgiving
Wk = 52, Christmas

In [18]:
date = datetime.strptime("2010-01-01", "%Y-%m-%d") # this is outside our time range
print(date.isocalendar()[1])
date = datetime.strptime("2010-01-04", "%Y-%m-%d")
print(date.isocalendar()[1])
date = datetime.strptime("2010-12-31", "%Y-%m-%d")
print(date.isocalendar()[1])


53
1
52


In [19]:
date = datetime.strptime("2011-01-07", "%Y-%m-%d")
print(date.isocalendar()[1])
date = datetime.strptime("2011-12-30", "%Y-%m-%d")
print(date.isocalendar()[1])

1
52


In [20]:
date = datetime.strptime("2012-01-06", "%Y-%m-%d")
print(date.isocalendar()[1])
date = datetime.strptime("2012-12-28", "%Y-%m-%d")
print(date.isocalendar()[1])

1
52


In [21]:
num_folds = 10
for i in range(num_folds):
    train = pd.read_csv(f'{path_to_data}/fold_{i+1}/train.csv')
    test = pd.read_csv(f'{path_to_data}/fold_{i+1}/test.csv')

    # Define start and end dates based on test data
    start_last_year = pd.to_datetime(test['Date'].min()) - timedelta(days=375)
    end_last_year = pd.to_datetime(test['Date'].max()) - timedelta(days=350)

    # Filter train data based on the defined dates and compute 'Wk' column
    tmp_train = train[(train['Date'] > str(start_last_year))
                      & (train['Date'] < str(end_last_year))].copy()
    tmp_train['Date'] = pd.to_datetime(tmp_train['Date'])
    tmp_train['Wk'] = tmp_train['Date'].dt.isocalendar().week
    tmp_train.rename(columns={'Weekly_Sales': 'Weekly_Pred'}, inplace=True)
    tmp_train.drop(columns=['Date', 'IsHoliday'], inplace=True)

    # Compute 'Wk' column for test data
    test['Date'] = pd.to_datetime(test['Date'])
    test['Wk'] = test['Date'].dt.isocalendar().week

    # Left join with the tmp_train data
    test_pred = test.merge(tmp_train, on=['Dept', 'Store', 'Wk'], how='left').drop(columns=['Wk'])

    # Fill NaN values with 0 for the Weekly_Pred column
    test_pred['Weekly_Pred'].fillna(0, inplace=True)


    # Write the output to CSV
    test_pred.to_csv(f'{path_to_data}/fold_{i+1}/mypred.csv', index=False)

In the implementation, when sourcing data from the corresponding week in the previous year, I didn't precisely offset by 365 days from the test period's start or end. Instead, I slightly expanded the range (subtracting a bit more than 365 days from the starting date and a bit less from the ending date). This adjustment helps to circumvent potential misalignments arising from our week counting method.

Below is the performance.

In [22]:
wae = myeval()

In [23]:
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

	2262.422
	1787.081
	1779.052
	1716.117
	2400.395
	1696.900
	2086.967
	1750.283
	1719.887
	1680.956
1888.006


**Jianci Code:**

In Python, we are lucky: within our specific time range, each year consistently consists of 52 weeks and the weeks align perfectly:

Wk = 6, Super Bowl Wk = 36, Labor Day Wk = 47, Thanksgiving Wk = 52, Christmas

In [24]:
#fill in missed sales data for dept with no sales number for certain week or store
def data_processing(proj2_data_train):

    proj2_data_train['Date'] = pd.to_datetime(proj2_data_train['Date'])

    train_store = pd.DataFrame({'Store':proj2_data_train['Store'].unique().tolist()})
    train_dept = pd.DataFrame({'Dept':proj2_data_train['Dept'].unique().tolist()})
    train_date  = proj2_data_train.groupby(['Date','IsHoliday']).count().reset_index()[['Date','IsHoliday']]

    train_combo = train_store.merge(train_dept, how='cross').merge(train_date, how='cross')

    col =['Store', 'Dept', 'Date','IsHoliday']

    #not sure if needed
    proj2_data_train = pd.merge(train_combo,proj2_data_train, on=col, how = 'left' )

    proj2_data_train['Wk'] = proj2_data_train['Date'].dt.isocalendar().week
    proj2_data_train['Yr'] = proj2_data_train['Date'].dt.year

    train_wk = pd.DataFrame({'Wk':proj2_data_train['Wk'].unique().tolist()})

    proj2_data_train['weights'] = proj2_data_train['IsHoliday'].astype(int)*4
    proj2_data_train['weights'] = proj2_data_train['weights']+1

    proj2_data_train = pd.get_dummies(proj2_data_train, columns = ['Wk'] , drop_first=False)
    wk_nb = [item for item in proj2_data_train.columns if (item.startswith('Wk_'))]
    #print(wk_nb)
    for i in wk_nb+['IsHoliday']:
      proj2_data_train[i] = proj2_data_train[i].astype('int')

    #fill in missed sales data for dept with no sales number for certain week or store
    proj2_data_train['Wk'] = proj2_data_train['Date'].dt.isocalendar().week

    try:
      proj2_data_train['Weekly_Sales'] = proj2_data_train['Weekly_Sales'].fillna(0)
    except:
      print('no sales data')

    return proj2_data_train, train_store, train_dept, train_date


**Naive Guess**

In [25]:
proj2_data_train = pd.read_csv(f'{path_to_data}/fold_1/train.csv')
proj2_data_test = pd.read_csv(f'{path_to_data}/fold_1/test.csv')


proj2_data_train,train_store, train_dept, train_date= data_processing(proj2_data_train)
proj2_data_test, test_store, test_dept, test_date = data_processing(proj2_data_test)

no sales data


In [26]:
proj2_data_train['Weekly_Sales'].quantile(0.9999)

293912.3917869992

In [27]:
#proj2_data_train['Weekly_Sales'] = proj2_data_train['Weekly_Sales'].clip( upper=proj2_data_train['Weekly_Sales'].quantile(0.99999))
proj2_data_train['Weekly_Sales'] = proj2_data_train['Weekly_Sales'].clip( lower=0,upper=400000)

In [28]:
proj2_data_actual = pd.read_csv(f'{path_to_data}/test_with_label.csv')
proj2_data_actual,_,_,_ = data_processing(proj2_data_actual)

In [29]:
#proj2_data_train[(proj2_data_train['Store']==10) & (proj2_data_train['Dept']==72)]

In [30]:
proj2_data_train.columns

Index(['Store', 'Dept', 'Date', 'IsHoliday', 'Weekly_Sales', 'Yr', 'weights',
       'Wk_1', 'Wk_2', 'Wk_3', 'Wk_4', 'Wk_5', 'Wk_6', 'Wk_7', 'Wk_8', 'Wk_9',
       'Wk_10', 'Wk_11', 'Wk_12', 'Wk_13', 'Wk_14', 'Wk_15', 'Wk_16', 'Wk_17',
       'Wk_18', 'Wk_19', 'Wk_20', 'Wk_21', 'Wk_22', 'Wk_23', 'Wk_24', 'Wk_25',
       'Wk_26', 'Wk_27', 'Wk_28', 'Wk_29', 'Wk_30', 'Wk_31', 'Wk_32', 'Wk_33',
       'Wk_34', 'Wk_35', 'Wk_36', 'Wk_37', 'Wk_38', 'Wk_39', 'Wk_40', 'Wk_41',
       'Wk_42', 'Wk_43', 'Wk_44', 'Wk_45', 'Wk_46', 'Wk_47', 'Wk_48', 'Wk_49',
       'Wk_50', 'Wk_51', 'Wk_52', 'Wk'],
      dtype='object')

In [31]:
proj2_data_train_to_future = proj2_data_train
proj2_data_train_to_future['Yr'] = proj2_data_train['Yr']+1
proj2_data_train_to_future['naive_pred'] = proj2_data_train['Weekly_Sales']

proj2_data_actual = proj2_data_actual.merge(proj2_data_train_to_future[[ 'Yr','Wk','Store', 'Dept', 'naive_pred']], on=['Yr','Wk','Store','Dept'], how='left')

In [32]:
proj2_data_actual.to_csv('proj2_data_actual_naive.csv')

In [33]:
proj2_data_actual = proj2_data_actual[~(proj2_data_actual['naive_pred'].isna())]

In [34]:
naive_pred = sum(proj2_data_actual['weights'] * abs(proj2_data_actual['naive_pred'] - proj2_data_actual['Weekly_Sales'])) / sum(proj2_data_actual['weights'])
naive_pred

1614.7157949649152

**Linear Model**

In [35]:
proj2_data_train = pd.read_csv(f'{path_to_data}/fold_10/train.csv')
proj2_data_test = pd.read_csv(f'{path_to_data}/fold_10/test.csv')


proj2_data_train,train_store, train_dept, train_date= data_processing(proj2_data_train)
proj2_data_test, test_store, test_dept, test_date = data_processing(proj2_data_test)

no sales data


In [36]:
cov_mtx = proj2_data_train[[item for item in proj2_data_train.columns if (item.startswith('Wk_') or item == 'Weekly_Sales')]].astype('int').cov()



In [37]:
# why dup values???
cov_mtx =cov_mtx.drop_duplicates()[['Weekly_Sales']]
cov_mtx['abs'] = abs(cov_mtx['Weekly_Sales'])
cov_mtx= cov_mtx.sort_values(by= 'abs', ascending=False)

In [38]:
var_week = cov_mtx.index[1:30].tolist() #most important variables

In [39]:
iv = var_week+['Store','Dept','IsHoliday','Yr']

In [40]:
var_week

['Wk_51',
 'Wk_47',
 'Wk_50',
 'Wk_49',
 'Wk_4',
 'Wk_2',
 'Wk_3',
 'Wk_17',
 'Wk_38',
 'Wk_39',
 'Wk_12',
 'Wk_30',
 'Wk_52',
 'Wk_8',
 'Wk_37',
 'Wk_41',
 'Wk_1',
 'Wk_22',
 'Wk_20',
 'Wk_7',
 'Wk_42',
 'Wk_27',
 'Wk_29',
 'Wk_14',
 'Wk_43',
 'Wk_48',
 'Wk_11',
 'Wk_15',
 'Wk_6']

In [41]:
X = proj2_data_train[iv]
Y = proj2_data_train['Weekly_Sales']

In [42]:
import statsmodels.api as sm

In [43]:
X2 = sm.add_constant(X)
X2

,const,Wk_51,Wk_47,Wk_50,Wk_49,Wk_4,Wk_2,Wk_3,Wk_17,Wk_38,...,Wk_14,Wk_43,Wk_48,Wk_11,Wk_15,Wk_6,Store,Dept,IsHoliday,Yr
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,2010
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,2010
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,2010
3,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,2010
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492070,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,45,65,0,2012
492071,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,45,65,0,2012
492072,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,45,65,0,2012
492073,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,45,65,0,2012


In [44]:

est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     393.7
Date:                Sun, 20 Oct 2024   Prob (F-statistic):               0.00
Time:                        01:14:13   Log-Likelihood:            -5.5988e+06
No. Observations:              492075   AIC:                         1.120e+07
Df Residuals:                  492041   BIC:                         1.120e+07
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.016e+05   8.17e+04     -2.467      0.0

In [45]:
for i in iv:
  try:
    proj2_data_test[i] = proj2_data_test[i].astype('int')
  except:
   proj2_data_test [i] = 0

In [46]:
X_test = proj2_data_test

In [47]:
X_test['const'] = 1

In [48]:
X_test
X2_test = X_test[['const']+iv]

In [49]:
y_fitted = est2.predict(X2_test)

In [50]:
proj2_data_test['lm_pred'] = y_fitted

In [51]:
proj2_data_test

,Store,Dept,Date,IsHoliday,Yr,weights,Wk_36,Wk_37,Wk_38,Wk_39,...,Wk_7,Wk_27,Wk_29,Wk_14,Wk_48,Wk_11,Wk_15,Wk_6,const,lm_pred
0,1,1,2012-09-07,1,2012,5,1,0,0,0,...,0,0,0,0,0,0,0,0,1,13707.476565
1,1,1,2012-09-14,0,2012,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,12949.871794
2,1,1,2012-09-21,0,2012,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,12566.048001
3,1,1,2012-09-28,0,2012,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,12615.306649
4,1,1,2012-10-05,0,2012,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13853.318516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,45,78,2012-09-28,0,2012,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,9965.468951
28796,45,78,2012-10-05,0,2012,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11203.480818
28797,45,78,2012-10-12,0,2012,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,10389.143446
28798,45,78,2012-10-19,0,2012,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,10575.882707


In [52]:
proj2_data_actual = pd.read_csv(f'{path_to_data}/test_with_label.csv')
proj2_data_actual,_,_,_ = data_processing(proj2_data_actual)

In [53]:
proj2_data_actual = proj2_data_actual.merge(proj2_data_test[[ 'Date','Store', 'Dept', 'lm_pred']], on=[ 'Date','Store', 'Dept'], how='left')
proj2_data_actual = proj2_data_actual[~(proj2_data_actual['lm_pred'].isna())]

#linear model results
lm_pred = sum(proj2_data_actual['weights'] * abs(proj2_data_actual['lm_pred'] - proj2_data_actual['Weekly_Sales'])) / sum(proj2_data_actual['weights'])
lm_pred

13672.343348682536

**tree model**

In [54]:
proj2_data_train = pd.read_csv(f'{path_to_data}/fold_1/train.csv')
proj2_data_test = pd.read_csv(f'{path_to_data}/fold_1/test.csv')


proj2_data_train,train_store, train_dept, train_date= data_processing(proj2_data_train)
proj2_data_test, test_store, test_dept, test_date = data_processing(proj2_data_test)

no sales data


In [55]:
# Initialize the model

gb = GradientBoostingRegressor(n_estimators=500,random_state=42)

In [56]:
gb.fit(proj2_data_train[iv], proj2_data_train['Weekly_Sales'])

GradientBoostingRegressor(n_estimators=500, random_state=42)

In [57]:
for i in iv:
  try:
    proj2_data_test[i] = proj2_data_test[i].astype('int')
  except:
   proj2_data_test [i] = 0

In [58]:
y_pred_tree = gb.predict(proj2_data_test[iv])

In [59]:
y_pred_tree

array([21039.99240203, 21039.99240203, 21039.99240203, ...,
        -824.81466406,  -824.81466406, -1319.2248086 ])

In [60]:
proj2_data_test['tree_pred'] = y_pred_tree

In [61]:
proj2_data_actual = pd.read_csv(f'{path_to_data}/test_with_label.csv')
proj2_data_actual,_,_,_ = data_processing(proj2_data_actual)

In [62]:
proj2_data_actual = proj2_data_actual.merge(proj2_data_test[[ 'Date','Store', 'Dept', 'tree_pred']], on=[ 'Date','Store', 'Dept'], how='left')
proj2_data_actual = proj2_data_actual[~(proj2_data_actual['tree_pred'].isna())]

#linear model results
tree_pred = sum(proj2_data_actual['weights'] * abs(proj2_data_actual['tree_pred'] - proj2_data_actual['Weekly_Sales'])) / sum(proj2_data_actual['weights'])
tree_pred

5538.326311138702

**Naive Guess with Smooth**

In [63]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


from scipy import stats
from scipy.optimize import minimize_scalar

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression as lm
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import SplineTransformer
from sklearn.cluster import KMeans


np.random.seed(3113)

In [64]:
!pip install scikit-misc==0.3.1 #match the exact version from the reference, function output keywords slightly different
from skmisc.loess import loess
#from csaps import csaps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 41.9 MB/s eta 0:00:00


In [65]:
"""
1- fit a loess model y1=x1 with span = sp, and extract
   the corresponding residual vector
2- call S_lam to obrain the diagonal entries of S
3- compute LOO-CV and GCV
"""
# Helper function to retrieve the diagonal of the smoother matrix S
def S_lam(x, span):
# Fit a LOESS model with a fake y vector (diagonal entries of S are independent of y)
    n = len(x)
    S_diag = np.zeros(n)

    # Loop through each point and perturb the y values slightly
    for i in range(n):
        # Perturb the i-th point
        y_perturbed = np.zeros(n)
        y_perturbed[i] = 1

        # Fit LOESS model with perturbed y
        loess_fit = loess(x, y_perturbed, span=span)
        loess_fit.fit()

        # Predict values (fitted values will form one row of S)
        S_diag[i] = loess_fit.predict(x).values[i]

    return S_diag


In [66]:
def loo_cv(x, y, span):
    """
    Compute the Leave-One-Out Cross-Validation (LOO-CV) for a given span.

    Inputs:
    - x: an n-by-1 feature vector (input data)
    - y: target values
    - span: a numerical value representing the span for LOESS

    Output:
    - LOO-CV error
    """
    loess_fit = loess(x, y, span=span)
    loess_fit.fit()
    y_pred = loess_fit.predict(x).values
    residuals = y - y_pred
    S_diag = S_lam(x, span)

    # Compute LOO-CV error
    loo_errors = residuals / (1 - S_diag)
    loo_cv_error = np.mean(loo_errors**2)

    return loo_cv_error

def gcv(x, y, span):
    """
    Compute the Generalized Cross-Validation (GCV) for a given span.

    Inputs:
    - x: an n-by-1 feature vector (input data)
    - y: target values
    - span: a numerical value representing the span for LOESS

    Output:
    - GCV error
    """
    loess_fit = loess(x, y, span=span)
    loess_fit.fit()
    y_pred = loess_fit.predict(x).values
    residuals = y - y_pred
    S_diag = S_lam(x, span)

    # Effective degrees of freedom
    edf = np.sum(S_diag)

    # Compute GCV error
    gcv_error = np.mean(residuals**2) / ((1 - edf / len(x))**2)

    return gcv_error

def find_optimal_span(x, y, span_values):
    """
    Find the optimal span based on CV and GCV.

    Inputs:
    - x: an n-by-1 feature vector (input data)
    - y: target values
    - span_values: a list of span values to test

    Output:
    - cv_errors: list of LOO-CV errors for each span
    - gcv_errors: list of GCV errors for each span
    - best_span: span with the lowest CV and GCV errors
    """
    cv_errors = []
    gcv_errors = []

    for span in span_values:
        cv_error = loo_cv(x, y, span)
        gcv_error = gcv(x, y, span)
        cv_errors.append(cv_error)
        gcv_errors.append(gcv_error)

    # Find the best span based on the minimum CV error
    best_span = span_values[np.argmin(cv_errors)]

    return cv_errors, gcv_errors, best_span


In [67]:
def plot_results(x, y, best_span):
    """
    Plot the original data points and fitted curve using the optimal span.

    Inputs:
    - x: feature vector
    - y: target values
    - best_span: the optimal span for LOESS
    """
    loess_fit = loess(x, y, span=best_span)
    loess_fit.fit()

    # Generate a finer grid for plotting a smooth curve
    #x_fine = np.linspace(min(x), max(x), 500)
    y_fitted = loess_fit.predict(x).values

    # Plot original data points and fitted curve
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, label='Original data', color='gray', s=50, facecolor='none')
    plt.plot(x, y_fitted, label='Fitted LOESS Curve', color='blue', linewidth=2, linestyle='--')

    # True curve: f(x) = sin(12*(x+0.2))/(x+0.2)
    #true_curve = np.sin(12 * (x_fine + 0.2)) / (x_fine + 0.2)
    #plt.plot(x_fine, true_curve, label='True Curve', color='grey')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.title(f'LOESS with optimal span {best_span}')
    plt.show()



In [68]:
np.around(np.linspace(0.1, 0.2, 15),3)

array([0.1  , 0.107, 0.114, 0.121, 0.129, 0.136, 0.143, 0.15 , 0.157,
       0.164, 0.171, 0.179, 0.186, 0.193, 0.2  ])

In [69]:
span_values = np.around(np.linspace(0.1, 0.9, 15),2)
span_values[2:3]

array([0.21])

In [70]:

span_values = np.around(np.linspace(0.1, 0.9, 15),2)
best_smooth_naive_pred= 9999999
span = 0.33
folder = 1
#for folder in range(1,11):

for span in span_values:

  proj2_data_train = pd.read_csv(f'{path_to_data}/fold_1/train.csv')
  proj2_data_test = pd.read_csv(f'{path_to_data}/fold_1/test.csv')

  #proj2_data_train = pd.read_csv(f'{path_to_data}/fold_{folder}/train.csv')
  #proj2_data_test = pd.read_csv(f'{path_to_data}/fold_{folder}/test.csv')

  proj2_data_train,train_store, train_dept, train_date= data_processing(proj2_data_train)
  proj2_data_test, test_store, test_dept, test_date = data_processing(proj2_data_test)
  train_to_future=pd.DataFrame()

  for s_i, store in enumerate(train_store['Store'].tolist()):
    for d_i, dept in enumerate(train_dept['Dept'].tolist()):
      tmp_train = proj2_data_train[(proj2_data_train['Store']==store) & (proj2_data_train['Dept']==dept)]
      y = tmp_train['Weekly_Sales'].values
      x = tmp_train['Date'].values



      # Define span values
      #span_values = np.around(np.linspace(0.2, 0.9, 15),2)


      # Find optimal span
      #cv_errors, gcv_errors, best_span = find_optimal_span(x, y, span_values)

      loess_fit = loess(x, y, span=span) # almost like a winzorization
      loess_fit.fit()
      y_fitted = loess_fit.predict(x).values

      tmp_train['Smooth_Weekly_Sales'] = y_fitted

      tmp_train_to_future = tmp_train
      tmp_train_to_future['Yr'] = tmp_train['Yr']+1
      tmp_train_to_future['naive_pred'] = tmp_train['Weekly_Sales']
      tmp_train_to_future['smooth_naive_pred'] = tmp_train['Smooth_Weekly_Sales']

      try:
        train_to_future = pd.concat([train_to_future, tmp_train_to_future])
      except:
        train_to_future = tmp_train_to_future

  proj2_data_actual = pd.read_csv(f'{path_to_data}/test_with_label.csv')
  proj2_data_actual, actual_store, actual_dept, actual_date = data_processing(proj2_data_actual)

  proj2_data_actual = proj2_data_actual.merge(train_to_future[[ 'Yr','Wk','Store', 'Dept','smooth_naive_pred']], on=['Yr','Wk','Store','Dept'], how='left')

  proj2_data_actual = proj2_data_actual[~(proj2_data_actual['smooth_naive_pred'].isna())]
  #naive prediction results
  smooth_naive_pred = sum(proj2_data_actual['weights'] * abs(proj2_data_actual['smooth_naive_pred'] - proj2_data_actual['Weekly_Sales'])) / sum(proj2_data_actual['weights'])
  if smooth_naive_pred < best_smooth_naive_pred:
    best_smooth_naive_pred = smooth_naive_pred
  print(f'in folder {folder}, when span = {span}, reach smooth_naive_pred = {smooth_naive_pred}')




no sales data
when span = 0.1, reach smooth_naive_pred = 1747.7127199611043
no sales data
when span = 0.16, reach smooth_naive_pred = 1823.432561496959
no sales data
when span = 0.21, reach smooth_naive_pred = 1859.2952570816371
no sales data
when span = 0.27, reach smooth_naive_pred = 1883.5709507466597
no sales data
when span = 0.33, reach smooth_naive_pred = 1911.7611965869337
no sales data
when span = 0.39, reach smooth_naive_pred = 1937.5346590936235
no sales data


KeyboardInterrupt: 

In [ ]:
for i in range(1,11):
  print(i)

In [ ]:
train_to_future

**SUMMARY**

In [ ]:

print(f'naive_pred: {naive_pred}')
print(f'tree_pred: {tree_pred}')
print(f'lm_pred: {lm_pred}')
print(f'smooth_naive_pred: {best_smooth_naive_pred}')